In [75]:
from connections import bb_session, teams_session
bb_s = bb_session()
ms_s = teams_session()

# Teams Creation
This is still very much a work in progress, but the script does the following:
* Pull membership information from Blackboard
* Create an Education Team, with instructors as owners
* Create private channels for each child course
* Add students to the Team.
* Add students to private channels.

In [97]:
# Pull membership information from Blackboard, adding in Microsoft ID
parentCourse = 'TST-Merge01'
memberships = bb_s.get(f'v1/courses/courseId:{parentCourse}/users', params={'expand':'user'}).json()['results']
memberships = [(member['user']['userName'], 
                bb_s.get(f"v2/courses/{member.get('childCourseId', member.get('courseId'))}").json()['courseId'],
                member['courseRoleId'],
                ms_s.get(f"users/{member['user']['userName']}@home.ku.edu").json()['id'])
                for member in memberships]

In [98]:
memberships

[('j944c412',
  'TST-Merge02',
  'Student',
  '27aa6ba6-efec-41d4-9751-c3c49e56e287'),
 ('j752h984',
  'TST-Merge02',
  'Student',
  '58e4e5f3-2326-420d-a73f-16f1b75b5128'),
 ('jmeston', 'TST-Merge01', 'Student', '88df618c-a588-4c2a-9b8a-47a3853fe7f0'),
 ('r684t861',
  'TST-Merge01',
  'Student',
  'f9193265-d10f-4651-8b23-70cda0f518b9'),
 ('m500d520',
  'TST-Merge01',
  'Instructor',
  'ea8452fa-bfe9-42ea-8543-9f7bacd248a9')]

In [27]:
#Pull course information and create team.
course_info = bb_s.get(f'v2/courses/courseId:{parentCourse}').json()
name = course_info['name']
team = {
    "template@odata.bind": "https://graph.microsoft.com/beta/teamsTemplates('educationClass')",
    "displayName": name,
    "description": 'A demo education teams site',
    "owners@odata.bind": [
        "https://graph.microsoft.com/beta/users('ea8452fa-bfe9-42ea-8543-9f7bacd248a9')"
    ]
}

r = ms_s.post("teams", json=team)
url = r.headers['Content-Location'].strip(")").strip("'").replace("('", "/")[1:]
team_info = ms_s.get(url).json()
print(team_info)

In [99]:
#Save team_id to add users and then channels
team_id = team_info['id']
print(team_id)

adf387ae-628b-443b-8c87-28b8742857fb


In [72]:
#Go through all Students in the class, and add to the team
for user in [member[0] for member in memberships if member[2] == 'Student']:
    ms_id = member[3]
    payload= {
            "@odata.type": "#microsoft.graph.aadUserConversationMember",
            "roles": [''],
            "user@odata.bind": f"https://graph.microsoft.com/beta/users/{ms_id}"
        }
    r = ms_s.post(f"teams/{team_id}/members",
        json=payload
    )
    if r.status_code == 201:
        print(f"Added {user} to team")
    else:
        print("Of course something went wrong")

Added j944c412 to team
Added j752h984 to team
Added jmeston to team
Added r684t861 to team


In [90]:
#Divide memberships into specific child courses
merge1members = [member for member in memberships if member[1] == 'TST-Merge01' or member[2] == 'Instructor']
merge2members = [member for member in memberships if member[1] == 'TST-Merge02' or member[2] == 'Instructor']

In [93]:
#Create private channel for first child course.
members = []
for membership in merge1members:
    if membership[2] == 'Student':
        roles = ["member"]
    else:
        roles = ["owner"]
    members.append({"@odata.type": "#microsoft.graph.aadUserConversationMember",
                "user@odata.bind": f"https://graph.microsoft.com/beta/users/{membership[3]}",
                 "roles":roles})

    channel = {
        "@odata.type": "#Microsoft.Teams.Core.channel",
        "displayName": 'TST-Merge01',
        "description": 'A private channel for one course',
        "membershipType": "private",
        "members": members,
    }

r = ms_s.post(f"teams/{team_id}/channels", json=channel)
print(r.json())

{'@odata.context': "https://graph.microsoft.com/beta/$metadata#teams('adf387ae-628b-443b-8c87-28b8742857fb')/channels/$entity", 'id': '19:4fe6eeb8d65548a2b112dcd94bc01c8d@thread.tacv2', 'createdDateTime': None, 'displayName': 'TST-Merge01', 'description': 'A private channel for one course', 'isFavoriteByDefault': None, 'email': '', 'webUrl': 'https://teams.microsoft.com/l/channel/19%3a4fe6eeb8d65548a2b112dcd94bc01c8d%40thread.tacv2/TST-Merge01?groupId=adf387ae-628b-443b-8c87-28b8742857fb&tenantId=3c176536-afe6-43f5-b966-36feabbe3c1a', 'membershipType': 'private', 'moderationSettings': None}


In [94]:
#Create private channel for second child course.
members = []
for membership in merge2members:
    if membership[2] == 'Student':
        roles = ["member"]
    else:
        roles = ["owner"]
    members.append({"@odata.type": "#microsoft.graph.aadUserConversationMember",
                "user@odata.bind": f"https://graph.microsoft.com/beta/users/{membership[3]}",
                 "roles":roles})

    channel = {
        "@odata.type": "#Microsoft.Teams.Core.channel",
        "displayName": 'TST-Merge02',
        "description": 'A private channel for one course',
        "membershipType": "private",
        "members": members,
    }

r = ms_s.post(f"teams/{team_id}/channels", json=channel)
print(r.json())

{'@odata.context': "https://graph.microsoft.com/beta/$metadata#teams('adf387ae-628b-443b-8c87-28b8742857fb')/channels/$entity", 'id': '19:6cc488f62e504b9a981a158f78be0275@thread.tacv2', 'createdDateTime': None, 'displayName': 'TST-Merge02', 'description': 'A private channel for one course', 'isFavoriteByDefault': None, 'email': '', 'webUrl': 'https://teams.microsoft.com/l/channel/19%3a6cc488f62e504b9a981a158f78be0275%40thread.tacv2/TST-Merge02?groupId=adf387ae-628b-443b-8c87-28b8742857fb&tenantId=3c176536-afe6-43f5-b966-36feabbe3c1a', 'membershipType': 'private', 'moderationSettings': None}
